# A Reasonable & Precise Solution for ANZbank
###### team intro

## First of all,We need to introduce our blueprint by 5 steps :
Step 1 : Load data & Split features based on numerical/category

Step 2 : Preprocessing for numerical & category

Step 3 : Some wonderful feature engineering

Step 4 : Model select with K-fold & feature importance 

Step 5 : Conclusion

## Import some usefull package 

## Step 1 :Load data & Split features based on numerical/category

## Step 2 : Preprocessing for numerical & category

## Step 3 : Some wonderful feature engineering

## Step 4 : Model select with K-fold & feature importance 

###### Now we wanna select a perfect model to fit the data finished above, but what kind of model is right for us? According to our data:
###### 1> its dimension is closed to normal-distribution after we do the log-op 
###### 2>this is a 2-classification Problem & Positive and negative samples are not balanced
###### 3>a lot of Disordered-Category-Features provided with big Information-Entropy
###### 4>no missing values & Infomation Values isn't high except "duration"
###### Thus, we select the LightGBM as our model, because SVM has a low performance when it learned from these not-high-IV features & Bagging method(Random Forest) maybe bad than Boosting when solve the multi-values-features 

In [1]:
%matplotlib inline
def plot_fea_importance(classifier,X_train):
    plt.figure(figsize=(10,12))
    name = "xgb"
    indices = np.argsort(classifier.feature_importances_)[::-1][:40]
    g = sns.barplot(y=X_train.columns[indices][:40],
                    x=classifier.feature_importances_[indices][:40],orient='h')
    g.set_xlabel("Relative importance", fontsize=12)
    g.set_ylabel("Features", fontsize=12)
    g.tick_params(labelsize=9)
    g.set_title(name + " feature importance")
    plt.show()

In [2]:
def evaluate_cv5_lgb(train_df, test_df, cols, test=False):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    y_test = 0
    oof_train = np.zeros((train_df.shape[0],))
    for i, (train_index, val_index) in enumerate(kf.split(train_df[cols])):
        X_train, y_train = train_df.loc[train_index, cols], train_df.y.values[train_index]
        X_val, y_val = train_df.loc[val_index, cols], train_df.y.values[val_index]
        lgb = LGBMClassifier(n_estimators=1000,
                            learning_rate=0.2,
                            num_leaves=30,
                            colsample_bytree=.8,
                            subsample=.9,
                            max_depth=7,
                            reg_alpha=.1,
                            reg_lambda=.1,
                            min_split_gain=.01,
                            min_child_weight=2,
                            silent=-1,
                            verbose=-1,)
        lgb.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_val, y_val)],
                early_stopping_rounds=100, eval_metric=['auc'], verbose=False)
        y_pred = lgb.predict_proba(X_val)[:,1]
        if test:
            y_test += lgb.predict_proba(test_df.loc[:, cols])[:,1]
        oof_train[val_index] = y_pred
        if i==0:
            plot_fea_importance(lgb,X_train)
    gc.collect()
    auc = roc_auc_score(train_df.y.values, oof_train)
    y_test /= 5
    print('5 Fold auc:', auc)
    return y_test

Wall time: 0 ns


## Step 5 : Conclusion

In [ ]:
%%time
train_len=18000
train,test=df[:train_len], df[train_len:]
cols = [col for col in train.columns if col not in ['id','y','y_','duration']]
y_pred=evaluate_cv5_lgb(train,test,cols,True)
print("Test Auc:%f"%roc_auc_score(test.y,y_pred))

###### Finally , Thanks for your reading & Best Wishes